In [3]:
import requests
import random
import numpy as np

word_site = "http://svnweb.freebsd.org/csrg/share/dict/words?view=co&content-type=text/plain"
response = requests.get(word_site)
WORDS = response.content.splitlines()
proper = [word for word in WORDS if word[0].isupper()]
not_proper = [word for word in WORDS if not word[0].isupper()]

class formWords:
    def __init__(self, proper, not_proper):
        self.proper = proper
        self.not_proper = not_proper
        
    def randomProper(self):
        return self.proper[np.random.randint(0,len(self.proper))]
    
    def randomNonProper(self):
        return self.not_proper[np.random.randint(0,len(self.not_proper))]
        
    def randomNumber(self,digits):
        return str(random.sample(range(10 ** (digits - 1), 10 ** (digits)), 1)[0])
    
    def randomPostfix(self):
        postfix = ['Street','Ave', 'Blvd', 'Lane', 'Court']
        return postfix[np.random.randint(1,len(postfix))]
    
    def randomEmailProvider(self):
        provider = ['gmail','yahoo', 'live', 'compuserve', 'aol']
        return provider[np.random.randint(1,len(provider))]
    
    def randomState(self):
        states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
        return states[np.random.randint(1,len(states))]
        
    def generateText(self, blog_length):
        form = self.getForm()
        blog = []
        for x in range(blog_length):
            choice = np.random.randint(1,50)
            if choice < 38:
                blog += [self.randomNonProper()]
            elif choice < 40:
                blog += [form['email']()]
            elif choice < 44:
                blog += [form['name']()]
            elif choice < 46:
                blog += [form['password']()]
            elif choice < 50:
                blog += [form['address']()]
    
        return " ".join(blog)

    def getForm(self):
        return {
            'email' : lambda : "Email: " + self.randomNumber(4) + self.randomNonProper().lower() + 
                "@" + self.randomEmailProvider() + ".com",
            'name' : lambda : "Name: " + self.randomProper() + " " + self.randomProper(),
            'password' : lambda : "Password: " + self.randomNumber(2) + self.randomNonProper() + 
                self.randomNumber(3) + self.randomProper().lower(),
            'address' : lambda : "Address: " + self.randomNumber(3) + " " + self.randomProper() + 
                " " + self.randomPostfix() + ", " + self.randomProper() + " " + self.randomState() + 
                ", " + self.randomNumber(5),
        }

words = formWords(proper, not_proper)

In [4]:
print words.getForm()['email']()
print words.getForm()['name']()
print words.getForm()['password']()
print words.getForm()['address']()

#This is the data variable
data = words.generateText(10000)
print data[:100]

Email: 6755lunate@live.com
Name: Gina Fuji
Password: 84seminal992kettering
Address: 637 Hudson Blvd, Della HI, 51107
Name: Upton McKnight vengeance scattergun florican smart grouch pyrite Name: Bergman Bolivia acropol


### Pull all the emails from the data variable

In [5]:
import re
email = re.findall('\s+Email: (\w*@\w*.\w*)', data)
print email[0:5]

['1922ahem@live.com', '5046porphyry@compuserve.com', '6305common@live.com', '6629chipboard@live.com', '7445eye@yahoo.com']


### Pull all the names from the data variable

In [6]:
name = re.findall('\s+Name: (\w* \w*)', data)
print name[0:5]

['Bergman Bolivia', 'Barr Jorge', 'Luther Chinamen', 'Whittier Santiago', 'Manchuria Vasquez']


### Pull all the passwords from the data variable

In [7]:
password = re.findall('\s+Password: (\w*)', data)
print password[0:5]

['37runaway102beauregard', '52sedge143vichy', '17downside499burnside', '70lumbar521buchenwald', '63ignorant484boise']


### Pull all the addresses from the data variable

In [8]:
address = re.findall('\s+Address: (\d{3} \w*\s\w*\,\s\w*\s\w*\,\s\d+)', data)
print address[0:5]

['231 Davis Court, Bengal MS, 68969', '715 Clark Court, Flagler MN, 66710', '926 Adler Court, Argentinian NV, 26938', '726 Burch Court, Audubon OR, 30547', '894 Annie Blvd, Carnegie CO, 13741']


### Which state had the most addresses in your data?

In [10]:
import pandas as pd
states = []
for x in address:
    states += re.findall('(..).......$', x)
pd.Series(states).value_counts()

NM    25
MN    24
CO    24
MT    23
IN    23
MD    23
NV    21
AZ    21
TX    21
NY    20
AR    20
OR    20
IL    20
PA    19
UT    19
NE    19
DC    19
AK    19
WV    18
SD    18
WA    18
VT    17
RI    16
HI    16
NJ    16
MS    16
GA    16
ND    16
NC    15
VA    15
OK    15
ID    15
DE    14
OH    14
KY    14
SC    14
MA    13
LA    13
CT    13
FL    12
WI    12
IA    12
CA    11
WY    11
TN    11
ME    11
KS    11
MI    10
MO    10
NH     9
dtype: int64

### Do we have any famous people in our data? 
#### Check to see if there are any user names that are also city/street names

In [11]:
# [re.findall( '\w*$', x)[0] for x in name]

firstName =  [x.split(" ")[0] for x in name]
lastName =  [x.split(" ")[1] for x in name]

city = [x.split(" ")[-3] for x in address]
street = [x.split(" ")[1] for x in address]

for y in city:
    for x in name:
        if (x.split(" ")[0] == y) | (x.split(" ")[1] == y):
            print "Name: ", x, " City: ", y
for y in street:
    for x in name:
        if (x.split(" ")[0] == y) | (x.split(" ")[1] == y):
            print "Name: ", x, " Street: ", y

Name:  Bengal Webster  City:  Bengal
Name:  McGregor Audubon  City:  Audubon
Name:  Buxton Ganges  City:  Buxton
Name:  Quinn Christopher  City:  Quinn
Name:  Conant Allis  City:  Allis
Name:  Antarctica Baja  City:  Antarctica
Name:  Carthage Columbus  City:  Carthage
Name:  Carthage Sherrill  City:  Carthage
Name:  Carolinian Franz  City:  Carolinian
Name:  Dodson Anita  City:  Anita
Name:  Fuchsia Andorra  City:  Andorra
Name:  BEMA Tyson  City:  Tyson
Name:  Agway Suzanne  City:  Agway
Name:  Mekong Odin  City:  Odin
Name:  Gaberones Garvey  City:  Gaberones
Name:  Ehrlich Deneb  City:  Ehrlich
Name:  Vance Arkansan  City:  Arkansan
Name:  Cornell Krieger  City:  Krieger
Name:  Wilkins Sanford  City:  Wilkins
Name:  Athabascan Liz  City:  Athabascan
Name:  Hugh U  City:  Hugh
Name:  Jeff Galapagos  City:  Galapagos
Name:  Phoenicia Galapagos  City:  Galapagos
Name:  Jacob Thebes  City:  Jacob
Name:  Jacob Wisconsin  City:  Jacob
Name:  Fleming Saviour  City:  Fleming
Name:  Leland 

### What is the value count of the zipcodes in your addresses?

In [12]:
pd.Series([x.split(" ")[-1] for x in address]).value_counts()

90726    2
52016    2
71364    2
38229    1
98585    1
81343    1
52083    1
31764    1
59561    1
16717    1
19686    1
14334    1
25438    1
49070    1
25538    1
28555    1
71700    1
69905    1
12230    1
78759    1
42732    1
40355    1
69560    1
56502    1
68726    1
30449    1
87150    1
82638    1
55980    1
79578    1
        ..
58475    1
27516    1
58721    1
79638    1
13351    1
19820    1
40505    1
32109    1
79119    1
75072    1
86340    1
85676    1
74603    1
61367    1
63543    1
98280    1
66200    1
11636    1
91095    1
31157    1
14664    1
60097    1
22299    1
83499    1
39853    1
44962    1
53984    1
81073    1
17291    1
14177    1
dtype: int64